In [ ]:
import pandas as pd

df = pd.read_csv("/content/train.csv")
df1 = pd.read_csv("/content/test.csv")


print(df.shape, df1.shape)

(630000, 13) (270000, 12)


In [ ]:

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder


def encoding_df(data):
    ordinal_encoder = OrdinalEncoder()
    ordinal_encoded = ordinal_encoder.fit_transform(data.iloc[:, [8, 10, 11]])
    ordinal_df = pd.DataFrame(
        ordinal_encoded,
        columns=ordinal_encoder.get_feature_names_out(data.columns[[8, 10, 11]]),
    )

    ohe = OneHotEncoder(sparse_output=False)
    encoded = ohe.fit_transform(data.iloc[:, [2, 3, 6, 9]])
    encoded_df = pd.DataFrame(
        encoded, columns=ohe.get_feature_names_out(data.columns[[2, 3, 6, 9]])
    )

    input_data = pd.concat([data.iloc[:, [1, 4, 5, 7]], ordinal_df, encoded_df], axis=1)

    return input_data

In [ ]:
x_train = encoding_df(df)
x_test = df.iloc[:, -1]
y_train = encoding_df(df1)

x_train.shape, x_test.shape, y_train.shape

((630000, 24), (630000,), (270000, 24))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor

# reg = GradientBoostingRegressor()

# param_grd = {
#     "learning_rate": [0.01, 0.02],
#     "loss": ["squared_error", "absolute_error", "huber"],
#     "n_estimators": [100, 120],
#     "max_features": ["sqrt", "log2"]
# }

# grid_search = GridSearchCV(estimator=reg, param_grid=param_grd, cv=3, scoring='neg_root_mean_squared_error')

# grid_search.fit(x_train, x_test)

In [ ]:
# print("Best parameters:", grid_search.best_params_)
# grid_search

In [ ]:
# reg = AdaBoostRegressor().fit(x_train, x_test)

In [ ]:
# y_pred = reg.predict(x_train.iloc[:2000, :])
# y_true = x_test.iloc[:2000]

# from sklearn.metrics import root_mean_squared_error
# rmse = root_mean_squared_error(y_true, y_pred)

# print(reg)
# print(f"RMSE: {rmse}")

In [ ]:
# pred_output = reg.predict(y_train)

# pd.DataFrame({"id": df1["id"], "exam_score": pred_output}).to_csv(
#     "/content/output4.csv", index=False
# )

## using XGB

In [ ]:
!pip install xgboost -q

In [ ]:
x_train = encoding_df(df)
x_test = df.iloc[:, -1]
y_train = encoding_df(df1)

x_train.shape, x_test.shape, y_train.shape

((630000, 24), (630000,), (270000, 24))

In [ ]:
from xgboost import XGBRegressor

# create model instance
bst = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.09, subsample=0.69, colsample_bytree=0.8) ##  reducing Subsample helps in RMSE
# fit model
bst.fit(x_train, x_test)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.09, eval_metric=None,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None, ...)

In [ ]:
y_pred = bst.predict(x_train.iloc[:2000, :])
y_true = x_test.iloc[:2000]

from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y_true, y_pred)

print(f"RMSE: {rmse}")

RMSE: 8.302307739913735


In [ ]:
pred_output = bst.predict(y_train)

pd.DataFrame({"id": df1["id"], "exam_score": pred_output}).to_csv(
    "/content/output_new.csv", index=False
)

In [ ]:
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 18.1 MB/s eta 0:00:00


In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

X_train, X_valid, y_train, y_valid = train_test_split(
    x_train, x_test, test_size=0.2, random_state=42
)


def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 800, 1000),
        "max_depth": trial.suggest_int("max_depth", 4, 8),
        "eta": trial.suggest_float("eta", 0.01, 0.2, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 0.8),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.9),
        "eval_metric": "rmse",
        "random_state": 42,
    }

    model = xgb.XGBRegressor(**params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False,
    )

    preds = model.predict(X_valid)
    rmse = root_mean_squared_error(y_valid, preds)
    return rmse


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best RMSE:", study.best_value)
print("Best params:", study.best_params)

In [ ]:
# create model instance
bst = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
# fit model
bst.fit(x_train, x_test)

## through Torch

In [ ]:
import torch.nn as nn

# Define the model
model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
X = encoding_df(df).to_numpy()        # shape: (N, 24)
y = df.iloc[:, -1].to_numpy()         # shape: (N,)

print(X.shape, y.shape)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, shuffle=True
)

(630000, 24) (630000,)


In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

print(X_train.shape)  # (≈441000, 24)
print(y_train.shape)  # (≈441000, 1)

print(X_test.shape)   # (≈189000, 24)
print(y_test.shape)   # (≈189000, 1)


torch.Size([441000, 24])
torch.Size([441000, 1])
torch.Size([189000, 24])
torch.Size([189000, 1])


/tmp/ipython-input-2409255340.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
/tmp/ipython-input-2409255340.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test  = torch.tensor(X_test,  dtype=torch.float32)
/tmp/ipython-input-2409255340.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
/tmp/ipython-input-2409255340.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.deta

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Sequential(
    nn.Linear(24, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 16),
    nn.ReLU(),
    nn.Linear(16, 4),
    nn.ReLU(),
    nn.Linear(4, 1)
)

# optimizer = optim.Adam(model.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)
mse_loss = nn.MSELoss()

In [ ]:
for epoch in range(500):
    optimizer.zero_grad()

    preds = model(X_train)
    mse = mse_loss(preds, y_train)
    rmse = torch.sqrt(mse)

    rmse.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}: RMSE = {rmse.item():.4f}")

Epoch 0: RMSE = 13.3876
Epoch 50: RMSE = 12.2313
Epoch 100: RMSE = 10.8918
Epoch 150: RMSE = 10.0082
Epoch 200: RMSE = 9.8486
Epoch 250: RMSE = 9.8288
Epoch 300: RMSE = 9.8191
Epoch 350: RMSE = 9.8126
Epoch 400: RMSE = 9.8078
Epoch 450: RMSE = 9.8040


In [ ]:
model.eval()

Sequential(
  (0): Linear(in_features=24, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=16, bias=True)
  (5): ReLU()
  (6): Linear(in_features=16, out_features=4, bias=True)
  (7): ReLU()
  (8): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
# converting csv to numpy array
X_new = encoding_df(df1).to_numpy()     # shape: (N, 24)
# convertig data to tensor format
X_new = torch.tensor(X_new, dtype=torch.float32)


with torch.no_grad():
    predictions = model(X_new)   # shape: (N, 1)

# prediction of test data
predictions = predictions.cpu().numpy().flatten()

# saving data in csv file
pd.DataFrame({"id": df1["id"], "exam_score": predictions}).to_csv(
    "/content/output6.csv", index=False
)